In [1296]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [1297]:
from google.colab import files
files.upload()

{}

In [1298]:
#Cargamos el archivo csv
df=pd.read_csv('DenveLim.csv')
#Eliminamos la columna unnamed
df= df.drop(['Unnamed: 0'], axis=1)
df.head(20)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,360,https://www.airbnb.com/rooms/360,20241230155646,2024-12-30,city scrape,Sit in the Peaceful Garden of the Chickadee Co...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,https://a0.muscache.com/pictures/monet/Select-...,666,...,4.96,5.00,5.00,5.00,4.92,2.0,2.0,0.0,0.0,2.35
1,364,https://www.airbnb.com/rooms/364,20241230155646,2024-12-30,city scrape,Lodo / RiNo LOFT via airport train,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",https://a0.muscache.com/pictures/11766413/a2c5...,783,...,4.81,4.95,4.96,4.65,4.71,1.0,1.0,0.0,0.0,0.46
2,590,https://www.airbnb.com/rooms/590,20241230155646,2024-12-30,city scrape,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I love the diversity of my neighborhood and it...,https://a0.muscache.com/pictures/d84d5bc6-f7cb...,933,...,4.90,4.93,4.95,4.76,4.85,2.0,0.0,0.0,0.0,3.85
3,592,https://www.airbnb.com/rooms/592,20241230155646,2024-12-30,city scrape,private,This room is in the basement. It does not hav...,,https://a0.muscache.com/pictures/ba522ff9-84c9...,933,...,4.90,4.95,4.95,4.82,4.87,2.0,0.0,0.0,0.0,0.93
4,1940,https://www.airbnb.com/rooms/1940,20241230155646,2024-12-30,city scrape,Baker Studio: Private-full kitchen-W/D-walkable,Private studio with separate entrance in histo...,Walking through the Baker historical neighborh...,https://a0.muscache.com/pictures/miso/Hosting-...,2150,...,4.98,4.98,4.98,4.90,4.88,1.0,1.0,0.0,0.0,2.20
5,21745,https://www.airbnb.com/rooms/21745,20241230155646,2024-12-30,city scrape,Alexandra's King Bed Room Long-term Only,Thank you for visiting my King Bed Room site! ...,"I love my Uptown neighborhood, which is within...",https://a0.muscache.com/pictures/25195156-c9fc...,990,...,4.73,4.82,4.93,4.86,4.73,1.0,0.0,0.0,0.0,1.32
6,31503,https://www.airbnb.com/rooms/31503,20241230155646,2024-12-31,city scrape,Highland Park Guest House,HIGHLAND PARK GUEST HOUSE,,https://a0.muscache.com/pictures/ecd97638-3f98...,135298,...,4.98,4.95,4.98,4.88,4.87,1.0,1.0,0.0,0.0,1.02
7,39405,https://www.airbnb.com/rooms/39405,20241230155646,2024-12-30,city scrape,LoHi Secret Garden at Mulberry at Denver Cottages,Enjoy our oasis in the city and stay at one of...,For those who are interested in our local brew...,https://a0.muscache.com/pictures/c1686c33-4efa...,666,...,4.94,4.98,4.97,4.86,4.86,2.0,2.0,0.0,0.0,4.15
8,90307,https://www.airbnb.com/rooms/90307,20241230155646,2024-12-30,city scrape,Comfy King Bed (Park Hill),A private guest room and bathroom are located ...,Coffee/bakery market and restaurant are all at...,https://a0.muscache.com/pictures/hosting/Hosti...,489506,...,4.94,4.97,4.96,4.93,4.92,3.0,1.0,0.0,0.0,1.81
9,154999,https://www.airbnb.com/rooms/154999,20241230155646,2024-12-31,city scrape,Midland Lofts 209: In the Heart of Downtown,This condo is right in the heart of Downtown D...,,https://a0.muscache.com/pictures/hosting/Hosti...,745200,...,4.75,5.00,4.92,4.92,4.67,4.0,1.1,0.0,0.0,0.08


In [1299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1300]:
#Verificar los valores sin repetirse de una columna
unico1 = np.unique(df['review_scores_rating'])
unico1

array([4.65, 4.66, 4.67, 4.68, 4.69, 4.7 , 4.71, 4.72, 4.73, 4.74, 4.75,
       4.76, 4.77, 4.78, 4.79, 4.8 , 4.81, 4.82, 4.83, 4.84, 4.85, 4.86,
       4.87, 4.88, 4.89, 4.9 , 4.91, 4.92, 4.93, 4.94, 4.95, 4.96, 4.97,
       4.98, 4.99, 5.  ])

In [1301]:
#Convierto una variable a dicotómica
df['review_scores_rating']= df['review_scores_rating'].replace([0.0 <= 3.0], 0.0)
df['review_scores_rating']= df['review_scores_rating'].replace([3.0 <= 5.0], 1.0)
df['review_scores_rating']

,review_scores_rating
0,4.99
1,4.85
2,4.85
3,4.88
4,4.96
...,...
5146,4.90
5147,4.90
5148,4.90
5149,4.90


In [1302]:
df['host_identity_verified']= df['host_identity_verified'].replace([0.0 <= 3.0], 0.0)
df['host_identity_verified']= df['host_identity_verified'].replace([3.0 <= 5.0], 1.0)
df['host_identity_verified']

,host_identity_verified
0,t
1,t
2,t
3,t
4,t
...,...
5146,t
5147,t
5148,t
5149,t


In [1303]:
#Seleccion de variables dependientes
VD1 = df["host_is_superhost"].unique()
print(VD1)
VD2 = df["instant_bookable"].unique()
print(VD2)
VD3 = df["host_acceptance_rate"].unique()
print(VD3)
VD4 = df["room_type"].unique()
print(VD4)
VD5 = df["host_identity_verified"].unique()
print(VD5)
VD6 = df["has_availability"].unique()
print(VD6)
VD7 = df["review_scores_rating"].unique()
print(VD7)

['t' 'f' 'False']
['f' 't']
['100%' '80%' '97%' '98%' '71%' '75%' '61%' '55%' '84%' '90%' '86%' '95%'
 '94%' '93%' '78%' '92%' '0%' '77%' '57%' '54%' '79%' '67%' '50%' '89%'
 '44%' '99%' '60%' '91%' '88%' '68%' '76%' '62%' '33%' '96%' '42%' '83%'
 '38%' '59%' '70%' '87%' '66%' '31%' '85%' '65%' '82%' '46%' '40%' '25%'
 '81%' '20%' '72%' '74%' '53%' '63%' '73%' '56%' '58%' '17%' '29%' '39%'
 '64%' '4%' '48%' '27%' '14%' '6%' '30%' '49%' '51%' '22%' '43%']
['Entire home/apt' 'Private room' 'Shared room' 'Hotel room']
['t' 'f']
['t' 'True']
[4.99 4.85 4.88 4.96 4.72 4.89 4.92 4.75 4.7  4.81 4.98 4.87 4.9  4.97
 4.86 4.82 4.91 4.84 5.   4.67 4.78 4.71 4.95 4.93 4.94 4.74 4.76 4.79
 4.65 4.73 4.69 4.83 4.8  4.66 4.77 4.68]


**Primer modelo de regresion logica.**

In [1304]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI= df[['price']]
VD1= df['host_is_superhost']

#Redefinimos las variables
X= VI
y= VD1

In [1305]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [1306]:
#Se escalan todos los datos
escalar = StandardScaler()

In [1307]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [1308]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [1309]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1310]:
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [1311]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred)
print('Matriz 1:')
print(M)

Matriz 1:
[[  0   0  31]
 [  0   0 677]
 [  0   0 838]]


In [1312]:
y_pred = np.delete(y_pred, 2)
y_pred = np.append(y_pred, 'f')

In [1313]:
np.unique(y_pred)

array(['f', 't'], dtype=object)

In [1314]:
y_test = y_test.replace({'False': 'f'})
y_test.unique()

array(['t', 'f'], dtype=object)

In [1315]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision 1:')
print(Pre)

Precision 1:
1.0


In [1316]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred)
print('Exactitud 1:')
print(Ex)

Exactitud 1:
0.5426908150064683


In [1317]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad 1:')
print(Sen)

Sensibilidad 1:
1.0


**Segundo modelo de regresion logistica.**

In [1318]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1319]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI2= df[['review_scores_rating']]
VD2= df['host_is_superhost']

#Redefinimos las variables
X= VI2
y= VD2

In [1320]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1321]:
#Realizamos una prediccion
y_pred2 = algoritmo.predict(X_test)
y_pred2

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [1322]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred2)
print('Matriz 2:')
print(M)

Matriz 2:
[[  0   4  39]
 [  0 104 545]
 [  0  75 779]]


In [1323]:
y_test = y_test.replace({'False': 'f'})
y_test.unique()

array(['f', 't'], dtype=object)

In [1324]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred2, average="binary", pos_label="t")
print('Precision 2:')
print(Pre)

Precision 2:
0.5715333822450477


In [1325]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred2)
print('Exactitud 2:')
print(Ex)

Exactitud 2:
0.573738680465718


In [1326]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad 2:')
print(Sen)

Sensibilidad 2:
0.15606936416184972


**Tercer modelo de regresion logistica.**

In [1327]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1328]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI3= df[['review_scores_rating', 'price', 'host_acceptance_rate']]
VD3= df['instant_bookable']

#Redefinimos las variables
X= VI3
y= VD3

In [1329]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

X_train['host_acceptance_rate'] = X_train['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0
X_test['host_acceptance_rate'] = X_test['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1330]:
#Realizamos una prediccion
y_pred3 = algoritmo.predict(X_test)
y_pred3

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [1331]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred3)
print('Matriz 3:')
print(M)

Matriz 3:
[[1120    7]
 [ 416    3]]


In [1332]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred3, average="binary", pos_label="t")
print('Precision 3:')
print(Pre)

Precision 3:
0.3


In [1333]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred3)
print('Exactitud 3:')
print(Ex)

Exactitud 3:
0.7263906856403622


In [1334]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad 3:')
print(Sen)

Sensibilidad 3:
0.9937888198757764


**Cuarto modelo de regresion logistica.**

In [1335]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI4= df[['price']]
VD4= df['instant_bookable']

#Redefinimos las variables
X= VI4
y= VD4

In [1336]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1337]:
#Realizamos una prediccion
y_pred4 = algoritmo.predict(X_test)
y_pred4

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], dtype=object)

In [1338]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred4)
print('Matriz 4:')
print(M)

Matriz 4:
[[1086    0]
 [ 460    0]]


In [1339]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred4, average="binary", pos_label="f")
print('Precision 4:')
print(Pre)

Precision 4:
0.702457956015524


In [1340]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred4)
print('Exactitud 4:')
print(Ex)

Exactitud 4:
0.702457956015524


In [1341]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred4, average="binary", pos_label="t")
print('Sensibilidad 4:')
print(Sen)

Sensibilidad 4:
0.0


**Quinto modelo de regresion logistica.**

In [1342]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI5= df[['number_of_reviews', 'price', 'host_total_listings_count']]
VD5= df['host_identity_verified']

#Redefinimos las variables
X= VI5
y= VD5

In [1343]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1344]:
#Realizamos una prediccion
y_pred5 = algoritmo.predict(X_test)
y_pred5

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [1345]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred5)
print('Matriz 5:')
print(M)

Matriz 5:
[[   0  219]
 [   0 1327]]


In [1346]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred5, average="binary", pos_label="t")
print('Precision 5:')
print(Pre)

Precision 5:
0.8583441138421734


In [1347]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred5)
print('Exactitud 5:')
print(Ex)

Exactitud 5:
0.8583441138421734


In [1348]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred5, average="binary", pos_label="f")
print('Sensibilidad 5:')
print(Sen)

Sensibilidad 5:
0.0


**Sexto modelo de regresion logistica.**

In [1349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1350]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI6= df[['host_acceptance_rate', 'review_scores_rating']]
VD6= df['host_identity_verified']

#Redefinimos las variables
X= VI6
y= VD6

In [1351]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

# Convierte las columnas de porcentaje (si las hay) a valores numéricos
X_train['host_acceptance_rate'] = X_train['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0
X_test['host_acceptance_rate'] = X_test['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1352]:
#Realizamos una prediccion
y_pred6 = algoritmo.predict(X_test)
y_pred6

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [1353]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred6)
print('Matriz 6:')
print(M)

Matriz 6:
[[   0  218]
 [   0 1328]]


In [1354]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred6, average="binary", pos_label="t")
print('Precision 6:')
print(Pre)

Precision 6:
0.8589909443725744


In [1355]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred6)
print('Exactitud 6:')
print(Ex)

Exactitud 6:
0.8589909443725744


In [1356]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred6, average="binary", pos_label="f")
print('Sensibilidad 6:')
print(Sen)

Sensibilidad 6:
0.0


**Septimo modelo de regresion logistica.**

In [1357]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1358]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI7= df[['accommodates', 'host_acceptance_rate', 'price']]
VD7= df['room_type']

#Redefinimos las variables
X= VI7
y= VD7

In [1359]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

# Convierte las columnas de porcentaje (si las hay) a valores numéricos
X_train['host_acceptance_rate'] = X_train['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0
X_test['host_acceptance_rate'] = X_test['host_acceptance_rate'].replace('%', '', regex=True).astype(float) / 100.0

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1360]:
#Realizamos una prediccion
y_pred7 = algoritmo.predict(X_test)
y_pred7

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      dtype=object)

In [1361]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred7)
print('Matriz 7:')
print(M)

Matriz 7:
[[1329    0    5    0]
 [   1    0    1    0]
 [ 163    0   45    0]
 [   1    0    1    0]]


In [1362]:
y_test = y_test.replace({'Private room': 'No Entire home/apt', 'Shared room': 'No Entire home/apt', 'Hotel room': 'No Entire home/apt'})
y_test.unique()

array(['Entire home/apt', 'No Entire home/apt'], dtype=object)

In [1363]:
y_pred7 = np.where(y_pred7 == 'Private room', 'No Entire home/apt', y_pred7)
np.unique(y_pred7)

array(['Entire home/apt', 'No Entire home/apt'], dtype=object)

In [1364]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred7, average="binary", pos_label="No Entire home/apt")
print('Precision 7:')
print(Pre)

Precision 7:
0.9038461538461539


In [1365]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred7)
print('Exactitud 7:')
print(Ex)

Exactitud 7:
0.890038809831824


In [1366]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred7, average="binary", pos_label="Entire home/apt")
print('Sensibilidad 7:')
print(Sen)

Sensibilidad 7:
0.9962518740629686


**Octavo modelo de regresion logistica.**

In [1367]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5151 entries, 0 to 5150
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5151 non-null   int64  
 1   listing_url                                   5151 non-null   object 
 2   scrape_id                                     5151 non-null   int64  
 3   last_scraped                                  5151 non-null   object 
 4   source                                        5151 non-null   object 
 5   name                                          5151 non-null   object 
 6   description                                   5151 non-null   object 
 7   neighborhood_overview                         5151 non-null   object 
 8   picture_url                                   5151 non-null   object 
 9   host_id                                       5151 non-null   i

In [1368]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
VI8= df[['number_of_reviews', 'bedrooms']]
VD8= df['room_type']

#Redefinimos las variables
X= VI8
y= VD8

In [1369]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [1370]:
#Realizamos una prediccion
y_pred8 = algoritmo.predict(X_test)
y_pred8

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      dtype=object)

In [1371]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
M = confusion_matrix(y_test, y_pred8)
print('Matriz 8:')
print(M)

Matriz 8:
[[1335    0    0    0]
 [   4    0    0    0]
 [ 201    0    0    0]
 [   6    0    0    0]]


In [1372]:
y_test = y_test.replace({'Private room': 'No Entire home/apt', 'Shared room': 'No Entire home/apt', 'Hotel room': 'No Entire home/apt'})
y_test.unique()

array(['Entire home/apt', 'No Entire home/apt'], dtype=object)

In [1373]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

Pre = precision_score(y_test, y_pred8, average="binary", pos_label="No Entire home/apt")
print('Precision 8:')
print(Pre)

Precision 8:
0.0


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1374]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

Ex = accuracy_score(y_test, y_pred8)
print('Exactitud 8:')
print(Ex)

Exactitud 8:
0.8635187580853816


In [1375]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

Sen = recall_score(y_test, y_pred8, average="binary", pos_label="Entire home/apt")
print('Sensibilidad 8:')
print(Sen)

Sensibilidad 8:
1.0
